In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%pip install faiss-gpu
%pip install LangChain
%pip install sentence-transformers
%pip install pandas
%pip install numpy
%pip install pyarrow
%pip install fastparquet
%pip install torch
%pip install catboost
%pip install ipywidgets
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1e3245a1b50edcd15e90e0f68cd9d099ab4961e4da56a75f80540f8dc13cb9d8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0

In [56]:
%pip install DateTime

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for DateTime from https://files.pythonhosted.org/packages/d3/63/dced85cf2970b5d363e501a10aa051e71caea3ad10a60b7a57794f43a4b8/DateTime-5.3-py3-none-any.whl.metadata
  Obtaining dependency information for zope.interface from https://files.pythonhosted.org/packages/4f/20/94d4f221989b4bbdd09004b2afb329958e776b7015b7ea8bc915327e195a/zope.interface-6.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 3.1 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
BANNER = """
MMWXkolccc:::::::::::::::::::::cclokXWMM
MXd,.                              .,dXM
Xc.                            ..... .cX
k.                            .',;,.. .k
d.    ....................... .',;,.. .d
d.    'dOOOOOOOOOOOOOOOOOOOkdc'.....  .d
d.    ,0MMMMMMMMMMMMMMMMMMMMMWO;.     .d
d.    ,0MMMXdcccccccccccclOWMMWd.     .d
d.    ,0MMMO'             cXMMWx.     .d
d.    ,0MMMXocccccccccccclkWMMWd.     .d
d.    ,0MMMMMWWWWWWWWMMWMMMMMWK:.     .d
d.    ,0MMMWK000000000XWMMMNOo,.      .d
d.    ,0MMM0;.........,xNMMXo.        .d
d.    ,0MMM0,          .lKMMWO;.      .d
d.    'kXXXk'           .;kXXXO:.     .d
x.    ..'''..             .''''..     .x
0,                                    ,0
Wk,.                                .,kW
MWKd:'............................':dKWM
MMMMNKOxxddddddddddddddddddddddxxOKNMMMM
"""

In [2]:
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import math
import pandas as pd
import requests
from urllib.parse import urlencode
import os
import numpy as np
from catboost import CatBoostRanker, Pool
from functools import partial
import json
import pyarrow.parquet as pq
import time




# Формирование базы векторов

In [3]:
seed = 42
np.random.seed(seed)

In [17]:
candidates = pd.read_parquet(
    './data/videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

In [18]:
candidates = candidates.sample(n=10_000_000, replace=False, random_state=seed)

In [6]:
candidates

,video_id,video_title
6479006,video_24972264,Как мы снимали в публичном доме!
1521150,video_22470853,Капкейки из цветного теста
641816,video_22923068,Отзыв о рюкзаке Vikgeo | Распаковка рюкзаков V...
10116582,video_5078872,Наша ежевика на 24.06.2022г. Дальнейшие подкор...
16286594,video_19600512,КАК НАБРАТЬ МЫШЕЧНУЮ МАССУ И СИЛУ.ПОШАГОВАЯ ИН...
...,...,...
22883335,video_11937921,Музыкальный салон 2023. Концерт в Румянцевском...
33792615,video_21861162,"А.В.Клюев - МЕДИТАЦИЯ, Практическая Часть / ВС..."
31548102,video_9425722,Плоды просвещения
434212,video_25456361,Ремонт пластиковых окон в новостройке Новоросс...


In [7]:
#candidates.to_csv('candidates.csv',sep=',',index=False)

In [8]:
corpus = candidates['video_title'].apply(lambda x: x.lower()).values
video_ids = candidates['video_id'].values
#del candidates

Созданеи бд

In [9]:
st_model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    device='cuda'
)

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [10]:


ind2videoid = {ind: video_id for ind, video_id in enumerate(video_ids)}

In [11]:
def create_db(use_formed_index = True):
  #длинна эмбединга модели на выходе
  d = 768
  #Создание пустой бд
  if not use_formed_index:
    #для сравнивания векторов будет использоваться косинусное сходство
    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))



  with open('ind2videoid_15.json', 'w+') as f:
    json.dump(ind2videoid, f, indent=4)

  batch_size = 100000
  num_batches = math.ceil(len(corpus) / batch_size)

  generated_cand_name = 'generated_candidates_15.parquet'

  if not use_formed_index:
    try:
        for i in range(num_batches):
            # формируем батч
            start, end = i * batch_size, (i + 1) * batch_size
            corpus_batch = corpus[start:end]

            # считаем вектора для всех предложений в батче
            embeddings = st_model.encode(
                  corpus_batch,
                  batch_size=1000,
                  show_progress_bar=True
            )
         

            # добавляем новые батч векторов в индекс и сохраняем его
            index.add_with_ids(embeddings, np.array(range(start,end)))
            print(f'batch: {i + 1} / {num_batches}')
            
            # чистим ОЗУ
            del embeddings
        write_index(index, 'candidates_15.index')

        

    except KeyboardInterrupt:
        print('Остановлено пользователем')
        try:
            del embeddings
        except:
            pass


In [11]:
start=time.time()
create_db(True)
end=time.time()

[0;31mKernelOutOfMemory[0m: Kernel ran out of memory and has been restarted. If the restart fails, restart the kernel from the Kernel menu.
If the error persists, try choosing a different configuration or optimizing your code.

In [ ]:
end-start

# Подбор кандидатов по базе векторов

In [12]:
def read_parquets():
    automarkup = pd.read_parquet(
      '/home/jupyter/datasphere/project/data/automarkup.parquet',
      engine='fastparquet'
    )
  
    automarkup = automarkup[~automarkup['query'].isna()]
    automarkup['query'] = automarkup['query'].apply(lambda x: x.lower())
  
    #чтение фичей с паркета + одновременный отсев до самой последней дате обновления фичей
    features_parquet = pq.ParquetFile('/home/jupyter/datasphere/project/data/features.parquet')
    features, filter_date = None, '2023-05-02'
  
    for batch in features_parquet.iter_batches():
        tmp = batch.to_pandas()
        if features is None:
            features = tmp[tmp['report_date'] == filter_date]
        else:
            features = pd.concat([
                features,
                tmp[tmp['report_date'] == filter_date]
            ], axis=0)
  
    return [automarkup,features]

In [13]:
automarkup, features = read_parquets()

In [14]:
search_cpu_index = read_index('candidates_15.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

def get_top_k_videos(queries, search_cpu_index, topk = 5):
  
  
    generated_cand_name = 'generated_candidates_15.parquet'
    qembeddings = st_model.encode(
        queries,
        batch_size=1000,
        show_progress_bar=True
    )
  
    distance, faiss_ind = search_cpu_index.search(qembeddings, topk)
    generated_cand = {
        'query': [],
        'video_id': []
    }
    for i, q in enumerate(queries):
        vids = faiss_ind[i]
        generated_cand['video_id'] += [ind2videoid[v] for v in vids]
        generated_cand['query'] += [q] * len(vids)
    generated_cand = pd.DataFrame(generated_cand)
    generated_cand.to_parquet(
        generated_cand_name,
        engine='fastparquet'
    )
    return generated_cand

In [15]:
def finall_sim_top(queries,search_cpu_index):
    generated_cand=get_top_k_videos(queries,search_cpu_index)
    generated_cand=generated_cand.merge(features,how='inner',left_on='video_id',right_on='video_id')
    generated_cand=generated_cand.merge(candidates,how='inner',left_on='video_id',right_on='video_id')
    return generated_cand

In [19]:
generated_cand=finall_sim_top(['Россия сейчас'],search_cpu_index)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
generated_cand

,query,video_id,report_date,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days,video_title
0,Россия сейчас,video_25643733,2023-05-02,2022-06-24 11:43:32,UGC,Лайфстайл,2022-07-28 17:05:26,0,83,0,0,0,0,0,52667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,This is Russia
1,Россия сейчас,video_20014178,2023-05-02,2021-09-29 08:38:42,UGC,Технологии и интернет,2022-01-17 11:46:34,0,0,0,0,0,0,0,7240122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Russia: Global focus
2,Россия сейчас,video_14816104,2023-05-02,2022-03-28 11:00:40,ПрофКонтент,Искусство,2022-04-03 17:28:34,0,1,0,0,0,0,0,157409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Познавай Россию
3,Россия сейчас,video_33228630,2023-05-02,2022-04-03 07:16:27,UGC,Наука,2022-04-03 07:53:00,0,0,0,0,0,0,0,43257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,История России


# Формирование датасета

In [ ]:
# очищаем ОЗУ
#del generated_cand
#del search_cpu_index
#del automarkup
#del st_model
#del qembeddings

In [21]:

data = pd.read_csv('/home/jupyter/datasphere/project/manualmarkup.csv', on_bad_lines='skip',sep=',')
data =data.dropna(axis=0)

In [22]:
marks=set(data['sentiment'].values)
marcks_to_int={'useful': 3, 'could_be_better': 2, 'exact' : 1, 'uselessly': 0, 'сlickbait': 4}

In [23]:
def v_channel_reg_and_pub_datetime(pub_datetime):
    datetime_now = datetime.now()
    try:
        pub_datetime = datetime.strptime(pub_datetime, "%Y-%m-%d %H:%M:%S")
        delta = datetime_now - pub_datetime
        return delta.days
    except ValueError:
        return None

category = list(set(list(features["v_category"])))
def v_category_to_type(v_channel_type):

    category_dict = {value: category for category, value in enumerate(category)}
    result = category_dict.get(v_channel_type)

    return result

chanell = list(set(list(features["v_channel_type"])))
def v_channel_type_to_type(v_channel_type):
    channel_type_mapping = {value: category for category, value in enumerate(chanell)}

    return channel_type_mapping.get(v_channel_type)

In [24]:
def create_dataset(data,features):
  
    full_df2=data.merge(features, how='inner',left_on='video_id',right_on='video_id')
    marks=set(data['sentiment'].values)
    full_df2['sentiment']=full_df2['sentiment'].map(lambda x: marcks_to_int[x]/len(marks))
    #запихиваем значение position в диапозон от 0 до 1
    #print(set(full_df2['position'].values))
    #full_df2['position']=full_df2['position']/len(set(full_df2['position'].values))
    #full_df2['vtop'] = full_df2['vtop'].fillna(0)
    full_df2["v_channel_type"] = full_df2["v_channel_type"].map(v_channel_type_to_type)
    full_df2["v_category"] = full_df2["v_category"].map(v_category_to_type)
    #full_df2["v_channel_reg_datetime"] = full_df2["v_channel_reg_datetime"].map(v_channel_reg_and_pub_datetime)
    #full_df2["v_pub_datetime"] = full_df2["v_pub_datetime"].map(v_channel_reg_and_pub_datetime)
  
    full_df2 = full_df2.drop(["v_year_views", "v_week_views", "v_day_views", "v_week_views",
                   "v_cr_click_dislike_30_days",	"v_cr_click_vtop_30_days",
                   "v_cr_click_long_view_30_days",	"v_cr_click_comment_30_days",
                   "v_cr_click_like_1_days",	"v_cr_click_vtop_1_days",
                   "v_cr_click_long_view_1_days",	"v_cr_click_comment_1_days",
                   "query_date", "report_date",  "v_cr_click_like_7_days",
                   "v_cr_click_dislike_7_days", "v_cr_click_vtop_7_days",
                   "v_cr_click_long_view_7_days", "v_cr_click_comment_7_days",
                   "v_cr_click_like_30_days", "v_cr_click_dislike_1_days"], axis=1)
  
    #объеденение видео в группы по запросам
    full_df2['group_id'] = full_df2.groupby(['query']).ngroup()
  
    return full_df2


In [25]:
full_df2 = create_dataset(data,features)

In [26]:
full_df2.columns

Index(['query', 'video_id', 'sentiment', 'v_channel_reg_datetime',
       'v_channel_type', 'v_category', 'v_pub_datetime', 'total_comments',
       'v_month_views', 'v_likes', 'v_dislikes', 'v_duration', 'group_id'],
      dtype='object')

# разбивка датасета для обучения

In [27]:
groups = pd.Series(full_df2['group_id'].unique())
permutation = groups.sample(frac=1, random_state=seed)
train_groups, val_groups, test_groups = np.split(
    permutation,
    [int(0.75 * len(permutation)), int(0.90 * len(permutation))]
)

In [28]:
train_df = full_df2[full_df2['group_id'].isin(train_groups)]
val_df = full_df2[full_df2['group_id'].isin(val_groups)]
test_df = full_df2[full_df2['group_id'].isin(test_groups)]

In [29]:
train_df.sample(3)

,query,video_id,sentiment,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_month_views,v_likes,v_dislikes,v_duration,group_id
1904,главные вопросы сексологу / оргазмы/ нормы /по...,video_11982384,0.0,2019-08-27 19:29:12,1,8,2022-03-24 08:04:33,0,0,0,0,11634,265
978,грешницы,video_1668129,0.0,2019-05-23 17:20:06,1,20,2019-05-26 07:52:56,0,2,0,0,751561,278
6234,лебедевский мост в казани,video_30366870,0.0,2012-11-24 18:49:19,1,27,2023-03-11 20:11:13,0,1,0,0,87537,473


In [30]:
train_df = train_df.sort_values('group_id')
val_df = val_df.sort_values('group_id')
test_df = test_df.sort_values('group_id')

In [31]:
metainfo_columns = ['query', 'video_id', 'group_id', 'v_channel_reg_datetime','v_pub_datetime','sentiment']

X_train = train_df.drop(metainfo_columns, axis=1)
y_train, g_train = train_df['sentiment'], train_df['group_id']

X_val = val_df.drop(metainfo_columns, axis=1)
y_val, g_val = val_df['sentiment'], val_df['group_id']

X_test = test_df.drop(metainfo_columns, axis=1)
y_test, g_test = test_df['sentiment'], test_df['group_id']

In [32]:
train = Pool(
    data=X_train.values,
    label=y_train.values,
    group_id=g_train.values,
    feature_names=X_train.columns.to_list()
)

val = Pool(
    data=X_val.values,
    label=y_val.values,
    group_id=g_val.values,
    feature_names=X_val.columns.to_list()
)

test = Pool(
    data=X_test.values,
    label=y_test.values,
    group_id=g_test.values,
    feature_names=X_test.columns.to_list()
)

# Обучение модели

In [33]:
task_type = 'GPU'
metric_period = 250

parameters = {
    'task_type': task_type,
    'verbose': False,
    'random_seed': seed,
    'loss_function': 'PairLogitPairwise:max_pairs=1000',
#'max_pairs': 1000,
    'learning_rate': 0.001,
    'l2_leaf_reg': 30,
    'iterations': 4000,
    'max_depth': 3,
}

In [34]:
model = CatBoostRanker(**parameters)
model = model.fit(
    train,
    eval_set=val,
    plot=True,
    use_best_model=True,
    metric_period=metric_period
)
model.save_model('ranker.ckpt')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Измерение метрик

In [35]:
def _metrics_at(at, model, pool, metric='NDCG'):
    metric = metric + f':top={at}'
    eval_metrics = model.eval_metrics(pool, metrics=[metric])
    best_metrics = {}
    for key in eval_metrics.keys():
        best_metrics[key] = eval_metrics[key][model.best_iteration_]
    return best_metrics


metrics_train_at = partial(
    _metrics_at,
    model=model,
    pool=train
)

metrics_val_at = partial(
    _metrics_at,
    model=model,
    pool=val
)

metrics_test_at = partial(
    _metrics_at,
    model=model,
    pool=test
)

In [36]:
metrics_train_at(1), metrics_train_at(5)

({'NDCG:top=1;type=Base': 0.6678192703743525},
 {'NDCG:top=5;type=Base': 0.7917404455264934})

In [37]:
metrics_val_at(1), metrics_val_at(5)

({'NDCG:top=1;type=Base': 0.7060185173256207},
 {'NDCG:top=5;type=Base': 0.8036120706662628})

In [38]:
metrics_test_at(1), metrics_test_at(5)

({'NDCG:top=1;type=Base': 0.7104810975014074},
 {'NDCG:top=5;type=Base': 0.8097261428820567})

In [39]:
# очищаем ОЗУ
#del model
#del full_df
#del train_df, val_df, test_df
#del train, val, test
#del X_train, y_train, g_train
#del X_val, y_val, g_val
#del X_test, y_test, g_test